In [1]:
import numpy as np # always need it
import scipy as sp # often use it
import pylab as pl # do the plots
import seaborn as sns # for pretty plots
import matplotlib.pyplot as plt # for pretty plots

import ot # ot 

In [2]:
def Convolution_Wasserstein(a, b, C, l):
    u = np.ones(len(a))
    K = np.exp(-C/l)
    for i in range(len(a)):
        v = b / (K.T @ u)
        u = a / (K @ v)
    T = np.diag(u) @ K @ np.diag(v)
    return T

In [6]:
def Wasserstein_barycentre(mu_i, alpha, Ht, a, ent_sharp = True, N = 200):
    k = len(mu_i)
    d = np.zeros(k)
    v = np.ones((k,k))
    w = np.ones((k,k))
    for j in range(N):
        mu = np.ones(k)
        for i in range(k):
            w[i] = mu_i[i]/Ht@(a*v[i])
            d[i] = v[i]*Ht@(a*w[i])
            mu = mu*d[i]**alpha[i]

        #entropic sharpening, optional
        if (ent_sharp):
            H_0 = H0(mu_i, a)
            mu = entropic_sharpening(mu, H_0, a)

        for i in range(k):
            v[i] = v[i]*mu/d[i]

    return mu




In [2]:
def entropic_sharpening(mu, H0, a):
    beta = 1
    if(H(mu, a) + np.transpose(a)*mu > H0 + 1 ):
        beta = sp.optimize.root(np.transpose(a)*mu**beta + H(mu**beta, a) - (1 + H0), beta > 0)
    return mu**beta

In [3]:
def H(mu, a):
    sum = 0
    for i in range(len(mu)):
        sum -= a[i] * mu[i] * np.log(mu[i])
    return sum

In [4]:
def H0(mu_i,a):
    for i in range(len(mu_i)):
        h = H(mu_i[i],a)
        if(h > H0):
            H0 = h 
    return h

In [8]:
def Wasserstein_propagation(V, E, V0, mu_V0, Ht, a, alpha, N = 200):
    m = len(V)
    k = len(E)
    m0,k0 = np.shape(mu_V0)
    v = np.ones((k,k0))
    w = np.ones((k,k0))
    mu = np.ones((m,k0))
    d = np.zeros((k,k0))
    for j in range(N):
        for i in range(m):
            if (V[i] in V0):
                mu[i] = V0(V0.index(V[i]))
                for l in range(k): #on parcourt les aretes
                    e = E[l]
                    if e[1] == V[i] :
                        w[l] = mu / Ht @ (a * v[l])
                    if e[0] == V[i] :
                        v[l] = mu / Ht @ (a * w[l])
            else:
                omega = 0
                for l in range(k):
                    e = E[l]
                    if(e[1]==V[i] or e[0]==V[i]):
                        omega += alpha[l]
                for l in range(k): #on parcourt les aretes
                    e = E[l]
                    if e[1] == V[i] :
                        d[l] = w[l] * Ht @ (a * v[l])
                    if e[0] == V[i] :
                        d[l] = v[l] * Ht @ (a * w[l])
                    mu[i] = mu[i] * d[l]**(alpha[l]/omega)
                for l in range(k): #on parcourt les aretes
                    e = E[l]
                    if e[1] == V[i] :
                        w[l] = w[l] * mu[i]/d[l]
                    if e[0] == V[i] :
                        v[l] = v[l] * mu[i]/d[l]
            

    return mu